## dataset

https://dspy-docs.vercel.app/docs/building-blocks/data

https://dspy-docs.vercel.app/docs/cheatsheet#dspy-dataloaders

In [125]:
# file_path = "../.data/mix_model/data/all_raretrible.txt"
file_path = "/home/azhao/projects/ai/train/liubin/mix_model/data/all_raretrible.txt"
data = open(file_path, "r").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:]), "", "", ""] for x in data]
print(data[0])

['感激', '好的,那我谢谢您了。', '', '', '']


In [126]:
import pandas as pd

# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=["label", "text", "pred_label", "score", "keywords"])

# 保存为 CSV 文件
cvs_file = "../.data/data.csv"
df.to_csv(cvs_file, index=False)

In [127]:
from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
dataset = dl.from_csv(
    cvs_file,
    fields=("label", "text", "score", "pred_label", "score", "keywords"),
    input_keys=("text",),
)
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Example({'label': '感激', 'text': '好的,那我谢谢您了。', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys={'text'})


In [128]:
print(dataset[0].get("label"))
print(dataset[0].get("text"))
print(dataset[0].labels())
print(dataset[0].inputs())

感激
好的,那我谢谢您了。
Example({'label': '感激', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys=None)
Example({'text': '好的,那我谢谢您了。'}) (input_keys={'text'})


In [ ]:
# 改用 stratified_sample 来取得 平均分布的数据集
splits = dl.train_test_split(dataset, train_size=20, test_size=50)
# splits = dl.train_test_split(dataset, train_size=20, test_size=50, stratify=dataset['label'])

train_dataset = splits["train"]
test_dataset = splits["test"]
print(len(train_dataset), len(test_dataset))
print(train_dataset[0])
print(test_dataset[0])

In [35]:
# 统计 label 分布, 其实我们希望 label 分布是均匀的，并随机分布
from collections import Counter

label_counts = Counter(item.get("label") for item in train_dataset)
print(label_counts)

label_counts = Counter(item.get("label") for item in test_dataset)
print(label_counts)

Counter({'中性': 7, '惊讶': 5, '感激': 4, '生气': 3, '抱怨': 1})
Counter({'中性': 17, '感激': 10, '生气': 8, '惊讶': 6, '焦急': 3, '高兴': 3, '抱怨': 3})


In [ ]:
sampled_example = dl.sample(dataset, n=20)  # `dataset` is a List of dspy.Example
print(len(sampled_example), sampled_example[0])

In [129]:
from collections import defaultdict
import random


def stratified_sample(dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


# 使用示例
train_set, test_set, sample_set = stratified_sample(dataset, "label", 20, 50, 100)

In [113]:
# 统计 label 分布, 其实我们希望 label 分布是均匀的，并随机分布
from collections import Counter

for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 5, '中性': 4, '生气': 3, '焦急': 2, '惊讶': 2, '高兴': 2, '抱怨': 2})
Counter({'高兴': 8, '感激': 7, '中性': 7, '焦急': 7, '惊讶': 7, '抱怨': 7, '生气': 7})
Counter({'感激': 3, '惊讶': 2, '中性': 1, '焦急': 1, '高兴': 1, '抱怨': 1, '生气': 1})


In [115]:
for ex in train_dataset:
    print(ex.get("label"), ex.get("text"))

抱怨 交诚意金了为什么都不回信息?
感激 麻烦你了谢谢[emoji053]
生气 你们这什么玩意,这售后问题解决的太**了
中性 其他没有了,我先在线提交材料试着解锁看看,谢谢。
感激 哦哦好的感谢
高兴 好,好,很好
感激 算了,放弃,谢谢你
生气 一块钱也坑
感激 没有啦、非常感谢
高兴 是啊哈哈哈
感激 哦哦,好吧,谢谢你哦
感激 拜拜,辛苦了,祝你新年好
中性 一直在冲我刚刚一看就没电了
惊讶 这个是最后一个退货?
感激 对的是这个,谢谢啦!
中性 可以延期是吧,好的
中性 那东西是一样的么?
惊讶 明明我插进去了
中性 那么麻烦,那算了,
中性 这个我下载不了,麻烦也一并发我邮箱,谢谢


## basic qa

In [116]:
import os

import dspy
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), override=True)
api_key = os.getenv("ONEAPI_API_KEY")
base_url = os.getenv("ONEAPI_BASE_URL")

# 测试要加 model_type ，不然因 model 不存在，会默认 text
turbo = dspy.OpenAI(
    api_key=api_key,
    model="glm-4-flash",
    api_base=base_url,
    model_type="chat",
    max_tokens=8000,
)

dspy.settings.configure(
    lm=turbo,
)

In [117]:
import dspy
from dsp import passages2text


class BasicQA(dspy.Signature):
    """sentiment analysis from text"""

    question = dspy.InputField(desc="list of input text", format=passages2text)
    answer = dspy.OutputField(
        desc="question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']",
        type=list,
    )

In [ ]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

samples = [x.text for x in sample_set]
# Call the predictor on a particular input.
pred = generate_answer(question=samples)

# Print the input and the prediction.
print(f"Question: {samples}")
print(f"Predicted Answer:\n{pred.answer}")

In [140]:
# 答案和问题对应起来
def update_sample(sample_set, answer):
    for sample, prediction in zip(sample_set, answer.split("\n")):
        sample["score"] = prediction.split(",")[1]
        sample["pred_label"] = prediction.split(",")[0].split(" ")[-1]


def sample_error_rate(sample_set):
    incorrect_count = sum(
        1 for sample in sample_set if sample["pred_label"] != sample["label"]
    )
    total_count = len(sample_set)
    return incorrect_count / total_count


update_sample(sample_set, pred.answer)
print(sample_error_rate(sample_set))

for sample in sample_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

0.32
Question: Example({'label': '感激', 'text': '好的。知道了。谢谢您', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '没有,谢谢,再见', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '行。谢谢了', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '哦,我再试下,谢谢', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '太忙了,客服太少了吧', 'score': ' 0.8', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '我这是无线流量怎样说欠费了', 'score': ' 0.7', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '就是因为现在没办法归还,又不使用', 'score': ' 0.7', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '是的,我收货地点写的是我的

In [132]:
turbo.inspect_history(n=1)




sentiment analysis from text

---

Follow the following format.

Question: list of input text
Answer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']

---

Question:
[1] «好的。知道了。谢谢您»
[2] «非常感谢公司这么好的售后服务»
[3] «谢谢你!不用了»
[4] «哦哦,谢谢[emoji029]»
[5] «嗯嗯,好的,太感谢了?»
[6] «不用了!谢谢你啦»
[7] «谢谢,我己经找到。麻烦你了亲»
[8] «是这个,谢谢了»
[9] «谢谢!没事儿。»
[10] «没有,谢谢,再见»
[11] «没有了。。非常感谢»
[12] «行。谢谢了»
[13] «嗯嗯,谢谢您»
[14] «哦,我再试下,谢谢»
[15] «之前我一直在同一机柜上扫码»
[16] «太忙了,客服太少了吧»
[17] «可是我的营业执照上有这个医疗器械的类目»
[18] «说和商品同送的»
[19] «我这是无线流量怎样说欠费了»
[20] «您好,查询一下»
[21] «你是真人还是电脑»
[22] «我借充电宝才发现我的充电宝在菏泽»
[23] «就是因为现在没办法归还,又不使用»
[24] «虹桥火车站,国家会展中心附近»
[25] «是的,我收货地点写的是我的单位,因为我们可能要出去耍哈。好的,谢谢?»
[26] «转债速度»
[27] «我十四号回家老家过年了估计都还没有收到货»
[28] «点错支付宝了。付不了款»
[29] «太慢了,外面零下呢»
[30] «可是客人着急啊»
[31] «好的,请尽快邮寄。谢谢。»
[32] «已付款,请尽快处理,谢谢»
[33] «这个单子很紧急»
[34] «你好我买的口红怎么还没发货呢?»
[35] «你好,我的为什么那么久还没到嘞»
[36] «麻烦您帮忙查一下,尽快安排发货,谢谢了»
[37] «你赶紧结束»
[38] «你们这个急人»
[39] «可是你们回款太慢了»
[40] «帮我催一催谢谢»
[41] «我的期末成绩为什么还查不

"\n\n\nsentiment analysis from text\n\n---\n\nFollow the following format.\n\nQuestion: list of input text\nAnswer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']\n\n---\n\nQuestion:\n[1] «好的。知道了。谢谢您»\n[2] «非常感谢公司这么好的售后服务»\n[3] «谢谢你!不用了»\n[4] «哦哦,谢谢[emoji029]»\n[5] «嗯嗯,好的,太感谢了?»\n[6] «不用了!谢谢你啦»\n[7] «谢谢,我己经找到。麻烦你了亲»\n[8] «是这个,谢谢了»\n[9] «谢谢!没事儿。»\n[10] «没有,谢谢,再见»\n[11] «没有了。。非常感谢»\n[12] «行。谢谢了»\n[13] «嗯嗯,谢谢您»\n[14] «哦,我再试下,谢谢»\n[15] «之前我一直在同一机柜上扫码»\n[16] «太忙了,客服太少了吧»\n[17] «可是我的营业执照上有这个医疗器械的类目»\n[18] «说和商品同送的»\n[19] «我这是无线流量怎样说欠费了»\n[20] «您好,查询一下»\n[21] «你是真人还是电脑»\n[22] «我借充电宝才发现我的充电宝在菏泽»\n[23] «就是因为现在没办法归还,又不使用»\n[24] «虹桥火车站,国家会展中心附近»\n[25] «是的,我收货地点写的是我的单位,因为我们可能要出去耍哈。好的,谢谢?»\n[26] «转债速度»\n[27] «我十四号回家老家过年了估计都还没有收到货»\n[28] «点错支付宝了。付不了款»\n[29] «太慢了,外面零下呢»\n[30] «可是客人着急啊»\n[31] «好的,请尽快邮寄。谢谢。»\n[32] «已付款,请尽快处理,谢谢»\n[33] «这个单子很紧急»\n[34] «你好我买的口红怎么还没发货呢?»\n[35] «你好,我的为什么那么久还没到嘞»\n[36] «麻烦您帮忙查一下,尽快安排发货,谢谢了»\n[37] «你赶紧结束»\n[38] «你们这个急

### COT

In [134]:
# 加上COT
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=samples)

# Print the input, the chain of thought, and the prediction.
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Thought: We will analyze each text individually based on the context and the tone of the language used, and then classify the sentiment accordingly. The sentiment classification includes ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']. We will also assign a score from 0 to 1.0 to represent the intensity of the sentiment.
Predicted Answer: [1] 中性, 0.5
[2] 感激, 0.9
[3] 感激, 0.8
[4] 感激, 0.7
[5] 感激, 0.8
[6] 感激, 0.8
[7] 感激, 0.9
[8] 感激, 0.9
[9] 感激, 0.9
[10] 中性, 0.5
[11] 感激, 0.9
[12] 中性, 0.5
[13] 感激, 0.8
[14] 中性, 0.5
[15] 中性, 0.5
[16] 抱怨, 0.8
[17] 中性, 0.5
[18] 中性, 0.5
[19] 抱怨, 0.7
[20] 中性, 0.5
[21] 中性, 0.5
[22] 中性, 0.5
[23] 抱怨, 0.7
[24] 中性, 0.5
[25] 感激, 0.8
[26] 中性, 0.5
[27] 抱怨, 0.7
[28] 抱怨, 0.7
[29] 抱怨, 0.8
[30] 抱怨, 0.8
[31] 感激, 0.9
[32] 感激, 0.9
[33] 焦急, 0.9
[34] 焦急, 0.8
[35] 焦急, 0.8
[36] 焦急, 0.9
[37] 生气, 0.8
[38] 生气, 0.8
[39] 抱怨, 0.8
[40] 焦急, 0.9
[41] 焦急, 0.8
[42] 抱怨, 0.7
[43] 惊讶, 0.9
[44] 抱怨, 0.8
[45] 惊讶, 0.9
[46] 惊讶, 0.9
[47] 生气, 0.8
[48] 生气, 0.8
[49] 惊讶, 0.9
[50] 惊讶, 0.9
[51] 生气, 0.8
[52] 抱怨, 0

In [135]:
# 这样可以把答案和问题对应起来
for sample, prediction in zip(sample_set, pred.answer.split("\n")):
    sample["score"] = prediction.split(",")[1]
    sample["pred_label"] = prediction.split(",")[0].split(" ")[-1]
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

Question: Example({'label': '感激', 'text': '好的。知道了。谢谢您', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '没有,谢谢,再见', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '行。谢谢了', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '感激', 'text': '哦,我再试下,谢谢', 'score': ' 0.5', 'pred_label': '中性', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '太忙了,客服太少了吧', 'score': ' 0.8', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '我这是无线流量怎样说欠费了', 'score': ' 0.7', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '就是因为现在没办法归还,又不使用', 'score': ' 0.7', 'pred_label': '抱怨', 'keywords': None}) (input_keys={'text'})
Question: Example({'label': '中性', 'text': '是的,我收货地点写的是我的单位,因为

In [136]:
turbo.inspect_history(n=1)




sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']

---

Question:
[1] «好的。知道了。谢谢您»
[2] «非常感谢公司这么好的售后服务»
[3] «谢谢你!不用了»
[4] «哦哦,谢谢[emoji029]»
[5] «嗯嗯,好的,太感谢了?»
[6] «不用了!谢谢你啦»
[7] «谢谢,我己经找到。麻烦你了亲»
[8] «是这个,谢谢了»
[9] «谢谢!没事儿。»
[10] «没有,谢谢,再见»
[11] «没有了。。非常感谢»
[12] «行。谢谢了»
[13] «嗯嗯,谢谢您»
[14] «哦,我再试下,谢谢»
[15] «之前我一直在同一机柜上扫码»
[16] «太忙了,客服太少了吧»
[17] «可是我的营业执照上有这个医疗器械的类目»
[18] «说和商品同送的»
[19] «我这是无线流量怎样说欠费了»
[20] «您好,查询一下»
[21] «你是真人还是电脑»
[22] «我借充电宝才发现我的充电宝在菏泽»
[23] «就是因为现在没办法归还,又不使用»
[24] «虹桥火车站,国家会展中心附近»
[25] «是的,我收货地点写的是我的单位,因为我们可能要出去耍哈。好的,谢谢?»
[26] «转债速度»
[27] «我十四号回家老家过年了估计都还没有收到货»
[28] «点错支付宝了。付不了款»
[29] «太慢了,外面零下呢»
[30] «可是客人着急啊»
[31] «好的,请尽快邮寄。谢谢。»
[32] «已付款,请尽快处理,谢谢»
[33] «这个单子很紧急»
[34] «你好我买的口红怎么还没发货呢?»
[35] «你好,我的为什么那么久还没到嘞»
[36] «麻烦您帮忙查一下,尽快安排发货,谢谢了»

"\n\n\nsentiment analysis from text\n\n---\n\nFollow the following format.\n\nQuestion: list of input text\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']\n\n---\n\nQuestion:\n[1] «好的。知道了。谢谢您»\n[2] «非常感谢公司这么好的售后服务»\n[3] «谢谢你!不用了»\n[4] «哦哦,谢谢[emoji029]»\n[5] «嗯嗯,好的,太感谢了?»\n[6] «不用了!谢谢你啦»\n[7] «谢谢,我己经找到。麻烦你了亲»\n[8] «是这个,谢谢了»\n[9] «谢谢!没事儿。»\n[10] «没有,谢谢,再见»\n[11] «没有了。。非常感谢»\n[12] «行。谢谢了»\n[13] «嗯嗯,谢谢您»\n[14] «哦,我再试下,谢谢»\n[15] «之前我一直在同一机柜上扫码»\n[16] «太忙了,客服太少了吧»\n[17] «可是我的营业执照上有这个医疗器械的类目»\n[18] «说和商品同送的»\n[19] «我这是无线流量怎样说欠费了»\n[20] «您好,查询一下»\n[21] «你是真人还是电脑»\n[22] «我借充电宝才发现我的充电宝在菏泽»\n[23] «就是因为现在没办法归还,又不使用»\n[24] «虹桥火车站,国家会展中心附近»\n[25] «是的,我收货地点写的是我的单位,因为我们可能要出去耍哈。好的,谢谢?»\n[26] «转债速度»\n[27] «我十四号回家老家过年了估计都还没有收到货»\n[28] «点错支付宝了。付不了款»\n[29] «太慢了,外面零下呢»\n[30] «可是客人着急啊»\n[31] «好的,请尽快邮寄。谢谢。»\n[32] «已付款,请尽快处理,谢谢»\n[33] «这个单子很紧急»\n[34] «你好我买的口红怎么还没发货呢?»

### BootstrapFewShot

In [147]:
from dspy.teleprompt import BootstrapFewShot

import dspy
from dsp import passages2text


class BasicQA(dspy.Signature):
    """sentiment analysis from text"""

    question = dspy.InputField(desc="list of input text", format=passages2text)
    answer = dspy.OutputField(
        desc="question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']",
        type=list,
    )


class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(BasicQA)

    # 这里的 forward 会被调用 Sample 的 inputs_keys 作为参数,
    # 比如那里的 text 在这里，需要作为参数传给 forward(text = sample.get("text"))
    def forward(self, text=None):
        return self.prog(question=text)


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    update_sample([example], pred.answer)
    return example["label"] == example["pred_label"]


# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_cot = teleprompter.compile(CoT(), trainset=train_set)

 35%|███▌      | 7/20 [00:18<00:33,  2.61s/it]

Bootstrapped 4 full traces after 8 examples in round 0.


In [156]:
text = "\n".join(samples)
pred = compiled_cot(text=text)

BadRequestError: Error code: 400 - {'error': {'message': '系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。 (request id: 2024071819313738640109360580156)', 'type': '', 'param': '', 'code': '1301'}}

In [ ]:
BadRequestError: Error code: 400 - {'error': {'message': '系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。 (request id: 2024071819313738640109360580156)', 'type': '', 'param': '', 'code': '1301'}}


In [153]:
turbo.inspect_history(n=1)




sentiment analysis from text

---

Follow the following format.

Question: list of input text
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']

---

Question: 我看下都吓我一跳
Reasoning: Let's think step by step in order to produce the answer. We need to analyze the sentiment of the given text. The text "我看下都吓我一跳" can be broken down as follows:

1. "我看下" - This part of the sentence indicates a sudden realization or discovery, which could be neutral or slightly surprising.
2. "都吓我一跳" - This phrase explicitly conveys a sense of surprise or shock.

Considering these elements, the sentiment of the text is likely to be '惊讶' (surprise).

Answer: 惊讶,0.8





'\n\n\nsentiment analysis from text\n\n---\n\nFollow the following format.\n\nQuestion: list of input text\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 [\'中性\', \'惊讶\', \'感激\', \'抱怨\', \'焦急\', \'生气\', \'高兴\']\n\n---\n\nQuestion: 我看下都吓我一跳\nReasoning: Let\'s think step by step in order to\x1b produce the answer. We need to analyze the sentiment of the given text. The text "我看下都吓我一跳" can be broken down as follows:\n\n1. "我看下" - This part of the sentence indicates a sudden realization or discovery, which could be neutral or slightly surprising.\n2. "都吓我一跳" - This phrase explicitly conveys a sense of surprise or shock.\n\nConsidering these elements, the sentiment of the text is likely to be \'惊讶\' (surprise).\n\nAnswer: 惊讶,0.8\x1b\n\n\n'

In [ ]:
update_sample(sample_set, pred.answer)
print(sample_error_rate(sample_set))

for sample in sample_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question: {sample}")

## evaluate

In [59]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_base = Evaluate(
    devset=test_dataset, num_threads=1, display_progress=True, display_table=5
)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_base(generate_answer, metric=metric)

  0%|          | 0/50 [00:03<?, ?it/s]

ERROR:dspy.evaluate.evaluate:2024-07-18T06:01:47.769639Z [error    ] Error for example in dev set: 		 'Example' object has no attribute 'answer' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 1  (0.0):   2%|▏         | 1/50 [00:03<02:29,  3.05s/it]

ERROR:dspy.evaluate.evaluate:2024-07-18T06:01:47.775280Z [error    ] Error for example in dev set: 		 'Example' object has no attribute 'answer' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 2  (0.0):   4%|▍         | 2/50 [00:03<02:26,  3.05s/it]

ERROR:dspy.evaluate.evaluate:2024-07-18T06:01:47.777839Z [error    ] Error for example in dev set: 		 'Example' object has no attribute 'answer' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 3  (0.0):   6%|▌         | 3/50 [00:03<02:23,  3.05s/it]

ERROR:dspy.evaluate.evaluate:2024-07-18T06:01:47.781370Z [error    ] Error for example in dev set: 		 'Example' object has no attribute 'answer' [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 0.0 / 4  (0.0):   6%|▌         | 3/50 [00:03<02:23,  3.05s/it]WARNING: Not all input fields were provided to module. Present: []. Missing: ['question'].


AttributeError: 'Example' object has no attribute 'answer'